In [1]:
import numpy as np
from numpy import asarray, array, ndarray
from scipy.signal import convolve
import math

In [2]:
randn = np.random.randn
import timeit
def time(x, h, mode='valid', method='auto', repeat=20, number=2):
    times = timeit.repeat(
        "convolve(x, h, mode='{}', method='{}')".format(mode, method), 
        "import numpy as np\n" +
        "from scipy.signal import convolve",
        repeat=repeat,
        number=number,
        globals={"x": x, "h": h}
    )
    return min(times) / number

In [3]:
import pandas as pd

def _get_data(n, a, ndim, rng):
    if ndim == 2:
        x = rng.randn(n, n)
        h = rng.randn(a, a)
        if mode == "valid":
            x = rng.randn(n, n // 2)
            h = rng.randn(a, a // 2)
    elif ndim == 1:
        x = rng.randn(n)
        h = rng.randn(a)
    else:
        raise ValueError("ndim")
    return x, h

def time_conv(n, a, mode="full", ndim=2, random_state=None):
    rng = np.random.RandomState(random_state)
    x, h = _get_data(n, a, ndim, rng)
        
    t_fft = time(x, h, mode=mode, method="fft")
    t_direct = time(x, h, mode=mode, method="direct")
    return {
        "fft_time": t_fft,
        "direct_time": t_direct,
        "mode": mode,
        "x.shape[0]": x.shape[0],
        "x.shape[1]": x.shape[1],
        "h.shape[0]": h.shape[0],
        "h.shape[1]": h.shape[1],
        "random_state": random_state
    }

In [4]:
sizes = np.logspace(np.log10(3), np.log10(300), num=50).astype("int")
print(sizes)
today = "2019-10-04"
from tqdm.notebook import tqdm

data = []
for mode in ["full", "valid", "same"]:
    for n in tqdm(sizes):
        for a in sizes:
            if a > n:
                break
            datum = time_conv(n, a, random_state=len(data), ndim=2, mode=mode)
            data.append(datum)
            if len(data) % 100 == 0:
                pd.DataFrame(data).to_parquet(f"out/{today}-conv2d.parquet", index=False)
pd.DataFrame(data).to_parquet(f"out/{today}-conv2d.parquet", index=False)

[  3   3   3   3   4   4   5   5   6   6   7   8   9  10  11  12  13  14
  16  17  19  21  23  26  28  31  34  37  41  45  50  55  60  66  73  80
  88  97 106 117 128 141 155 170 187 205 226 248 273 300]
